In [1]:
import duckdb
import pandas as pd
import re
username = "adtserapio"
query = f"""
    SET temp_directory = '/wynton/protected/scratch/{username}_temp_dir.tmp';
    SET preserve_insertion_order = false;
    SET memory_limit = '96GB';
    SET threads TO 4;
"""
duckdb.execute(query)

In [5]:
note_text = duckdb.read_parquet('/wynton/protected/project/ic/data/parquet/DEID_CDW_UCSF/note_text/*.parquet')
note_metadata = duckdb.read_parquet('/wynton/protected/scratch/note_metadata_2024-08-19/*.parquet')
imagingfact = duckdb.read_parquet('/wynton/protected/project/ic/data/parquet/DEID_CDW_UCSF/imagingfact/*.parquet')

In [6]:
query = """
    SELECT 
        COUNT(DISTINCT note_metadata.patientdurablekey) AS unique_patientdurablekey_count,
        COUNT(DISTINCT note_metadata.deid_note_key) AS unique_deid_note_key_count
    FROM note_metadata
    WHERE note_type IS NOT NULL AND note_type IN ('Imaging')
"""
rad_report_counts = duckdb \
.sql(query) \
.df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
rad_report_counts

,unique_patientdurablekey_count,unique_deid_note_key_count
0,1745595,34361924


In [8]:
n_rad_total = rad_report_counts["unique_deid_note_key_count"].iloc[0]
n_patients_total = rad_report_counts["unique_patientdurablekey_count"].iloc[0]

print(n_patients_total, n_rad_total)

1745595 34361924


In [10]:
query = """
    SELECT *
    FROM (
        SELECT *
        FROM note_text
        WHERE note_text LIKE '%ADDITIONAL HISTORY%'
    ) nt
    INNER JOIN (
        SELECT *
        FROM note_metadata
        WHERE note_type IN ('Imaging')
    ) nm
    ON nt.deid_note_key = nm.deid_note_key;
"""
rad_report_additional_history = duckdb \
.sql(query) \
.df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [31]:
rad_report_additional_history["additional_history"] = rad_report_additional_history["note_text"].str.extract(r'ADDITIONAL HISTORY:\s*(.*?)\s*COMPARISON:').fillna("")

excluded_values = {"none", "none.", "none available", "none available.", "none provided", "none provided.","n/", "n/a", "history of", "research study", "research study.", "none provided.", "*****", "as above", ".", "see above", "no", "non-", "per", "", "no available relevant prior"}

rad_report_filter_none = rad_report_additional_history[
    (rad_report_additional_history["additional_history"].apply(len) > 0) &
    (~rad_report_additional_history["additional_history"].str.strip().str.lower().isin(excluded_values))
]

In [53]:
print("n_patients: ", rad_report_filter_none["patientdurablekey"].nunique())

n_patients:  88097
